# Load Text

서로 다른 작가가 호머의 일리아드를 영어로 번역한 3개의 텍스트를 읽어 들여 적절하게 데이터 전처리를 수행하고 데이터 셋으로 생성한다.

In [1]:
import tensorflow as tf
print(tf.__version__)

#import tensorflow_datasets as tfds
import os

2.3.0


### Text 파일 준비하기

호머의 일리아드를 영어로 번역한 3개의 텍스트를 사용한다.

In [2]:
#DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
DIRECTORY_URL = os.environ['USERPROFILE'] + '\\.keras\\datasets'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

'C:\\Users\\aoneko\\.keras\\datasets'

3개의 텍스트 파일을 읽어들여 하나의 데이터셋으로 만든다.

In [3]:
data = []     # 텍스트
labels = []   # labels 0=cowper, 1=derby, 2=butler
for i, file_name in enumerate(FILE_NAMES):
    with open(os.path.join(parent_dir, file_name)) as f:
        lines = f.readlines()
        cls = [i] * len(lines)
        data.extend(lines) 
        labels.extend(cls)

data[0], labels[0]

("癤풞chilles sing, O Goddess! Peleus' son;\n", 0)

### Tokenizer

* Tokenizer API를 사용하여 해당 문서를 token으로 분리한다.
* texts_to_sequences() 로 해당 단어를 숫자로 encode한다.
* pad_sequences()를 사용하면 해당 단어의 sequence의 길이를 모두 동일하게 맞출 수 있다.

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

def tokenize(lang):
    lang_tokenizer = Tokenizer()
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [5]:
tensor, tokenizer = tokenize(data)

tensor[0]

array([10414,  5198,   381,   257,   429,    21,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

`index_word()`  
tokenize 결과를 다시 단어 시퀀스로 변환하는 것도 가능하다.

In [ ]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [6]:
convert(tokenizer, tensor[0])

10414 ----> 癤풞chilles
5198 ----> sing
381 ----> o
257 ----> goddess
429 ----> peleus'
21 ----> son


##  tf.data 데이터 세트 만들기

딥러닝 모형에 입력으로 사용할 수 있게 텍스트 데이터를 데이터셋으로 만든다.

In [7]:
BUFFER_SIZE = len(tensor)
BATCH_SIZE = 64
vocab_size = len(tokenizer.word_index)

print(vocab_size)

15537


In [8]:
dataset = tf.data.Dataset.from_tensor_slices((tensor, labels)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [9]:
next(iter(dataset))

(<tf.Tensor: shape=(64, 19), dtype=int32, numpy=
 array([[  12,  368,  116, ...,    0,    0,    0],
        [  14,   39,  267, ...,    0,    0,    0],
        [  36,  100,   20, ...,    0,    0,    0],
        ...,
        [1040,    8, 3912, ...,    0,    0,    0],
        [   5,  799, 1365, ...,    0,    0,    0],
        [   5, 1303,  908, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(64,), dtype=int32, numpy=
 array([0, 2, 2, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 1, 1, 2, 2, 1,
        0, 2, 1, 2, 2, 0, 2, 1, 0, 1, 0, 0, 2, 1, 1, 1, 1, 0, 2, 0, 1, 1,
        2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0, 0, 1])>)